In [69]:
import pandas as pd
from collections import Counter
import numpy as np
from matplotlib.table import table
from sklearn.neighbors import KernelDensity
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid_spec
import jinja2
import re
import tabulate
import ridgeplot as rg
import nbformat
from sympy import true

import seaborn as sns
import plotly.io as pio
pio.renderers.default = "notebook"
from IPython.display import IFrame
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
#a

In [70]:
#Justin Li
# Basically, we're just trying to clean the dataset up so that the street names are more consistent.
df = pd.read_csv("2012-24_Crash_Events.csv")
print(df.columns)
column_nan_count = df['OnStreet'].isnull()
print(column_nan_count.sum())
column_unknown_count = df[(df['OnStreet'] == 'Unknown')]
print(column_nan_count.count())

df["OnStreet"] = df["OnStreet"].fillna("Unknown").astype(str)
column_nan_count = df['OnStreet'].isnull()

print(column_nan_count.sum())
column_unknown_count = df[(df['OnStreet'] == 'Unknown')]
print(column_nan_count.count())
df_streetsort = df.sort_values(by=['OnStreet'], inplace=True,)




/tmp/ipykernel_17434/2334159630.py:3: DtypeWarning:

Columns (0: CaseNumber) have mixed types. Specify dtype option on import or set low_memory=False.



Index(['CaseNumber', 'CaseYear', 'CrashDate', 'CrashTime',
       'CrashTimeFormatted', 'CrashTypeCde', 'CrashType', 'CollisionType',
       'CrashSeverity', 'LightCondition', 'WeatherCondition',
       'RoadwayAccessControlCde', 'RoadwayCharacteristic',
       'RoadSurfaceCondition', 'TrafficControl', 'TrafficWay',
       'MaxInjurySeverity', 'NumberOfFatalities', 'NumberOfInjuries',
       'NumberOfSeriousInjuries', 'NumberOfOtherInjuries', 'NumberOfVehicles',
       'PoliceDept', 'ReportingAgency', 'Precinct',
       'CommercialVehicleCrashInd', 'PPDRCode', 'NonReportable',
       'ReviewedIndicator', 'DMVInsertDate', 'CountyFIPS', 'CountyName',
       'MuniFIPS', 'CityTownName', 'OnStreet', 'ClosestCrossStreet',
       'MasterIntersectionId', 'IntersectionIndicator', 'ReferenceMarker',
       'DistanceFromIntersection', 'DirectionFromIntersection', 'UTMEasting',
       'UTMNorthing', 'NonPublicWayCode', 'ACCESS_CONTROL', 'DIVIDED',
       'FUNCTIONAL_CLASS', 'MAINT_JURISDICTION_TYP

In [71]:
onstreetnames = pd.unique(df['OnStreet'])
onstreetnames = onstreetnames.tolist()


with open("Street_names.txt", "w") as output:
    output.write(str(onstreetnames))


In [72]:
abbrev = {
    'ST': 'STREET',
    'RD': 'ROAD',
    'AVE': 'AVENUE',
    'AV': 'AVENUE',
    'BLVD': 'BOULEVARD',
    'LN': 'LANE',
    'DR': 'DRIVE',
    'CIR': 'CIRCLE',
    'CT': 'COURT',
    'PKWY': 'PARKWAY',
    'PKY': 'PARKWAY',
    'PL': 'PLACE',
    'TER': 'TERRACE',
    'TRL': 'TRAIL',
    'WAY': 'WAY',
    'HWY': 'HIGHWAY',
    'ALY': 'ALLEY',
    'AL': 'ALLEY',
    'CRES': 'CRESCENT',
    'KNL': 'KNOLL',
    'KN': 'KNOLL',
    'PIKE': 'PIKE',
    'MNR': 'MANOR',
    'GRV': 'GROVE',
    'HL': 'HILL',
    'TL': 'TOWN LINE',
    'EXT': 'EXTENSION',
    'RTE': 'STATE ROUTE',
    'RT': 'STATE ROUTE',
    'XING': 'CROSSING',
    'XNG': 'CROSSING',
    'RDG': 'RIDGE',
}

directions = {
    'N': 'NORTH',
    'S': 'SOUTH',
    'E': 'EAST',
    'W': 'WEST',
}

ordinals = {
    '1ST': 'FIRST',
    '2ND': 'SECOND',
    '3RD': 'THIRD',
    '4TH': 'FOURTH',
    '5TH': 'FIFTH',
    '6TH': 'SIXTH',
    '7TH': 'SEVENTH',
    '8TH': 'EIGHTH',
    '9TH': 'NINTH',
    '10TH': 'TENTH',
}

directional_terms = r'\b( NORTH| SOUTH| EAST| WEST|NORTHBOUND|SOUTHBOUND|EASTBOUND|WESTBOUND|NB|SB|EB|WB)\b'

In [73]:
# Grok generated this cell's code and the stuff immediately above. Generally speaking it takes the name, makes it uppercase, and does a bunch of regex to sub in the abbrevs. shown above.
# There are some concerns from the resulting code -- eg, Abbot street vs. Abbott street. I'll ultimately need Xinyi to take a look, make sure those aren't the same streets.
# Also a lot of streets are just numbers. I'm not sure what those might be, but I'm thinking it's a code for State Routes or Interstates? But iunno.
def standardize_street(name):

    firstname = name
    name = name.upper().strip()

    # Split concatenated suffixes (e.g., 'greyst' -> 'GREY ST')
    #suffixes = '|'.join(map(re.escape, list(abbrev.keys()) + list(abbrev.values())))
    #name = re.sub(r'([A-Z\d]+)(' + suffixes + r')$', r'\1 \2', name)

    # Replace ordinals
    for k, v in ordinals.items():
        name = re.sub(r'\b' + re.escape(k) + r'\b', v, name)

    # Handle interstates (I 490, I-490 → INTERSTATE 490)
    name = re.sub(r'\bI\s*-?\s*(\d+)\b', r'INTERSTATE \1', name)

    # Handle NY routes (NY104 → STATE ROUTE 104)
    name = re.sub(r'\bNY(\d+[A-Z]?)\b', r'STATE ROUTE \1', name)

    # Handle routes and common typos (expand RT, etc.)
    name = re.sub(r'\[ROUTE\]\s*(\d+)', r'STATE ROUTE \1', name)
    name = re.sub(r'\bRT\s*(\d+[A-Z]?)\b', r'STATE ROUTE \1', name)
    name = re.sub(r'STATE HWY\s*(\d+[A-Z]?)', r'STATE ROUTE \1', name)
    name = re.sub(r'STATE RTE\s*(\d+[A-Z]?)', r'STATE ROUTE \1', name)
    name = re.sub(r'STHY\s*(\d+[A-Z]?)', r'STATE ROUTE \1', name)

    # Remove exits (e.g., "EXIT 23 STATE ROUTE 390" → "STATE ROUTE 390")
    name = re.sub(r'EXIT\s*\d+\s*', '', name)

    # Remove ramps (e.g., "RAMP TO INTERSTATE 490" → "INTERSTATE 490")
    name = re.sub(r'RAMP\s*(TO|FROM)?\s*', '', name)

    # Remove directional terms
    name = re.sub(directional_terms, '', name)
    name = re.sub(r'\s+', ' ', name).strip()  # Clean extra spaces

    # Standardize words (directions expanded earlier, but now removed if present)
    words = name.split()
    new_words = []
    for word in words:
        if word in abbrev:
            new_words.append(abbrev[word])
        elif word in directions:
            new_words.append(directions[word])
        else:
            new_words.append(word)
    newname = ' '.join(new_words)
    if newname == 'AVENUE':
        print(firstname)
    return newname

In [74]:

#Because the above code doesn't catch stuff like "390" being State Route 390, I had to handcode this.
#'104', '15A', '251', '31F', '33', '383', '390', '441', '490', '590'

def numberlookup(num):
    match num:
        case "104":
            return "STATE ROUTE 104"
        case "15A":
            return "STATE ROUTE 15A"
        case "251":
            return "STATE ROUTE 251"
        case "31F":
            return "STATE ROUTE 31F"
        case "33":
            return "STATE ROUTE 33"
        case "65":
            return "STATE ROUTE 65"
        case "383":
            return "STATE ROUTE 33"
        #case "390":
        #   return  "STATE ROUTE 390"
        case "441":
            return "STATE ROUTE 441"
        case "490":
            return "INTERSTATE 490"
        case "590":
            return "INTERSTATE 590"
        case _:
            return num



In [75]:
df['OnStreet_standardized'] = df['OnStreet'].apply(standardize_street)
df['OnStreet_standardized'] = df['OnStreet_standardized'].apply(numberlookup)
df.to_csv('2012-24_Crash_Events_std_street_names_dataset.csv', index=False)

# Check reduction in uniques
print(f"Original uniques: {len(df['OnStreet'].unique())}")
print(f"Standardized uniques: {len(df['OnStreet_standardized'].unique())}")

Original uniques: 9509
Standardized uniques: 6331


In [76]:
df_streetsort = df.sort_values(by=['OnStreet_standardized'])

onstreetnames = pd.unique(df_streetsort['OnStreet_standardized'])
onstreetnames = onstreetnames.tolist()
onstreetnames
with open("Street_std_names.txt", "w") as output:
    output.write(str(onstreetnames))
